In [1]:
from xml.dom.minidom import parse
import xml.dom.minidom

from os import listdir, rename
from os.path import isfile, join
import time

import pandas as pd
import math as mt
from scipy.signal import savgol_filter



#pip install ipynb
import ipynb.fs.full.Weather_History as weather

ipynb.fs.full.Weather_History:150: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


# Funciones para ENRIQUECER los FICHEROS.
COMUN PARA TODOS LOS ORIGENES

In [2]:
def func_bearing(bc,bv):
    if bv+180 > 360:
        bv_mod = bv-180
    else:
        bv_mod = bv+180
    if bc > bv_mod:
        dif = bc-bv_mod
    else:
        dif = bv_mod-bc
        
    multiplicador_viento = mt.cos(dif/(180/mt.pi))
    #return(bc, bv, bv_mod, dif, multiplicador_viento)
    return(multiplicador_viento, dif)

def sync_bearing(ori_bearing):
    if ori_bearing < -90:
        new_bearing = 450+ori_bearing
    else:
        new_bearing = ori_bearing+90
    return new_bearing

def deteccion_tramos(input_df):
    prueba = input_df.iloc[1]["prueba"]

    tramos = [0]
    tramo = 0
    pendiente = 0
    pendiente_ant = 0

    tramos_100 = [0]
    tramo100 = 0
    acum_dist = 0
        
    lista_tramos = []
    tramo_inicial = 0
    power_acum = 0
    velocidad_aparente_acum = 0
    
    i = 1
    input_df.iloc[0,5] = 0
    input_df.iloc[1,5] = input_df.iloc[1,11]
    
    tiempo_parado = 0
    tiempo_parado_punto = 0
    
    print("Entramos en el While1 %d" % len(input_df.index))
    while i < len(input_df.index):
  
                
        #Calculo de tramos continuos
        
        #print("punto = %d y tramo = %d" % (i, tramo))
        pendiente = input_df.iloc[i]["new_ele"]-input_df.iloc[i-1]["new_ele"]

        #print("%d punto de %d" %(i,len(input_df.index)))

        if (input_df.iloc[i]["dist"]-input_df.iloc[i-1]["dist"])<1:
            tiempo_parado = tiempo_parado + (input_df.iloc[i]["timeoffset"]-input_df.iloc[i-1]["timeoffset"])
            tiempo_parado_punto = input_df.iloc[i]["timeoffset"]-input_df.iloc[i-1]["timeoffset"]
        
        if ((pendiente * pendiente_ant >=0) and ((i+1) != len(input_df.index))):
                
            tramos.append(tramo)
        else:
            #Cada tramo en 1 línea
            #Son intervalos cerrados por la izquierda y abiertos por la derecha
            #print("Cierra tramo")
            
            
            if ((i+1) != len(input_df.index)):
                distancia_tramo = input_df.iloc[i-1]["dist"]-input_df.iloc[tramo_inicial]["dist"]
                tiempo_tramo = (input_df.iloc[i-1]["timeoffset"]-input_df.iloc[tramo_inicial]["timeoffset"])-tiempo_parado
                desnivel_tramo = input_df.iloc[i-1]["new_ele"]-input_df.iloc[tramo_inicial]["new_ele"]
            else:
                distancia_tramo = input_df.iloc[i]["dist"]-input_df.iloc[tramo_inicial]["dist"]
                tiempo_tramo = input_df.iloc[i]["timeoffset"]-input_df.iloc[tramo_inicial]["timeoffset"]
                desnivel_tramo = input_df.iloc[i]["new_ele"]-input_df.iloc[tramo_inicial]["new_ele"]
            
            porcent_des = desnivel_tramo/distancia_tramo*100
                        
            
            
            sp_pwd = power_acum/tiempo_tramo
            sp_vel_aparente = velocidad_aparente_acum/distancia_tramo
            
            
            
            vel_calc = (distancia_tramo/tiempo_tramo)*3.6
            lista_tramos.append(\
                                (tramo, tramo_inicial,\
                                 i-1,sp_pwd,tiempo_tramo,\
                                 distancia_tramo,vel_calc,\
                                 sp_vel_aparente,porcent_des,\
                                 input_df.iloc[0]["dist"]+acum_dist,prueba))
            
            power_acum = 0
            velocidad_aparente_acum = 0
            tiempo_parado = 0
            
            #Cada Inicio o Fin de tramo en 1 línea
            tramo_inicial = i-1
            tramo +=1
            tramos.append(tramo)
        
        power_acum = power_acum + (input_df.iloc[i]["pwr"]*((input_df.iloc[i]["timeoffset"]-input_df.iloc[i-1]["timeoffset"])-tiempo_parado_punto))
        velocidad_aparente_acum = velocidad_aparente_acum +\
                                    (input_df.iloc[i]["viento_aparente"]*(input_df.iloc[i]["dist"]-input_df.iloc[i-1]["dist"]))
        tiempo_parado_punto = 0
            
        pendiente_ant = pendiente            
        
        #Calculo de tramos de 100m
        acum_dist = input_df.iloc[i-1]["dist"]
        tramo100 = mt.floor(acum_dist/100)
        tramos_100.append(tramo100)
        
        i +=1
          

    print("Salimos del While")
    print("tramos : %d continuos y %d de 100m" % (len(set(tramos)), len(set(tramos_100))))
    
   


    return lista_tramos, tramos, tramos_100


def enriquecimiento(list_track):
    #path='Analisis_Aerodinamica'
    #path='Entreno_preparatorio'
    #path='2017_05_13_Triatlón'
    path='20180406_LEVS_195_90_R_E'

    df = pd.DataFrame(list_track)
    df.columns = ["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp", "prueba"]
    df2 = df[df['lat'] >0]
    df2['pwr'].fillna(0, inplace=True)
    #df2.drop(['dist','cad','hr','spd'], axis=1, inplace=True)

    from geopy.distance import distance
    from geopy import Point

    i=1
    
    prueba = df2.iloc[1]["prueba"]
    info_prueba = prueba.split("_")
    ubicacion = info_prueba[1]
    fecha = info_prueba[0][:4]+"/"+str(int(info_prueba[0][4:6]))+"/"+str(int(info_prueba[0][-2:]))
    
    
    
    #Recupera el viento y la dirección automaticamente. Si se quiere ejecutar en manual para una prueba futura,
    #se debe comentar la línea e introducir manualmente dirección y velocidad
    viento, bv = weather.get_viento(ubicacion,fecha)
    #bv=90
    #viento=10
    
    
    viento_aparente=[]
    viento_aparente.append(0)

    wind=[]
    wind.append(0)

    mult=[]
    mult.append(0)

    dist_c=[]
    dist_c.append(0)

    ele_gain = []
    ele_gain.append(0)

    d=0
    e=0

    porc=[]
    porc.append(0)

    spd_c=[]
    spd_c.append(0)

    spd_prev=[]
    spd_prev.append(0)

    ele_prev=[]
    ele_prev.append(0)

    bearing=[]
    bearing.append(0)

    timeoffset_prev=[]
    timeoffset_prev.append(0)

    ini_timeoffset = df2.iloc[0]['timeoffset']
    time=[]
    time.append(0)


    while i < len(df2):
        #print("Punto %d" %i
        punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
        punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
        time.append(df2.iloc[i]['timeoffset']-ini_timeoffset)
        dist_c.append(distance(punto1, punto2).m)
        d = distance(punto1, punto2).m
        spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
        punto1 = df2.iloc[i-1]['alt']
        punto2 = df2.iloc[i]['alt']
        ele_gain.append((punto2)-(punto1))
        e = punto2-punto1
        porc.append(e*100/d)
        lona = df2.iloc[i-1]['lon']
        lata = df2.iloc[i-1]['lat']
        lonb = df2.iloc[i]['lon']
        latb = df2.iloc[i]['lat']
        
        X = mt.cos(lonb)*mt.sin(latb-lata)
        Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)

        #bearing.append(mt.atan2(X,Y)*57.2957795)  Se modifica por la siguiente línea que corrige estandariza el ángulo
        bearing.append(sync_bearing(mt.atan2(X,Y)*57.2957795))
        mult.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[0])
        wind.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[1])
        viento_aparente.append((func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[0])*viento)

        i+=1
    df2['dist_c']=dist_c
    df2['ele_gain']=ele_gain
    df2['porc']=porc
    df2['spd_c']=spd_c
    df2['bearing']=bearing
    df2['wind']=wind
    df2['mult']=mult
    df2['viento_aparente']=viento_aparente

    i=1
    while i < len(df2):
        spd_prev.append(df2.iloc[i-1]['spd_c'])
        ele_prev.append(df2.iloc[i-1]['alt'])
        timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
        i+=1
    df2['spd_prev']=spd_prev
    df2['ele_prev']=ele_prev
    df2['timeoffset_prev']=timeoffset_prev

    #aplicamos el suavizado de señal
    axis_y = df["alt"]
    axis_yhat = savgol_filter(axis_y, 51, 2)
    df2["new_ele"] = axis_yhat
    df2["time_seg"] = time

    
    lista_tramos, tramos, tramos_100 = deteccion_tramos(df2)
    

    df2["tramo"] = tramos
    df2["tramos_100"] = tramos_100


    df_tramos = pd.DataFrame(lista_tramos)
    df_tramos.columns= ["tramo", "tramo_inicial", "tramo_final","pwr","time_seg","dist","spd","viento_aparente","porc","dist_acum", "prueba"]
    #df_tramos.to_excel(result_tramos.xlsx')


    #AÑADIMOS NUEVAS VARIABLES DEPENDIENTES DE LA RUTA Y EL CICLISTA
    #altura = path.split('_')[2]
    #peso = path.split('_')[3]
    #tpo_bici = path.split('_')[4]
    #tpo_entreno = path.split('_')[5]

    df2['altura'] = path.split('_')[2]
    df2['peso'] = path.split('_')[3]
    df2['tpo_bici'] = path.split('_')[4]
    df2['tpo_entreno'] = path.split('_')[5] 

    return df2,df_tramos

## ANALISIS TRACK BASADO EN GPX para varios ficheros
#PASO_1
#Importamos los xml procedente de gpx que esten en la carpeta indicada por path
##########################################

In [3]:
ruta='input_files/'
ficheros = [f for f in listdir(ruta) if isfile(join(ruta, f)) & f.endswith(".gpx")]

result_list_tramos = []
result_list = []


for fichero in ficheros:
    #path='Analisis_Aerodinamica'
    #path='Entreno_preparatorio'
    #path='2017_05_13_Triatlón'
    #path='20180406_LEVS_195_90_R_E'
    path = join(ruta, fichero)
    
    print('Se inicia el procesado de %s' %fichero)

    DOMTree = xml.dom.minidom.parse(path)

    collection = DOMTree.documentElement

    trkpts = collection.getElementsByTagName("trkpt")
    list_track = []

    def to_dt(dato):
        t=pd.to_datetime(dato)
        return t.hour*3600+t.minute*60+t.second

    count=0

    for trkpt in trkpts:
        count +=1
        if trkpt.hasAttribute("lat"):
          lat = float(trkpt.getAttribute("lat"))
        if trkpt.hasAttribute("lon"):
          lon = float(trkpt.getAttribute("lon"))

        if len(trkpt.getElementsByTagName("ele"))==0:
            alt = ''
        else:
            alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("time"))==0:
            time = ''
        else:
            time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue

        if len(trkpt.getElementsByTagName("gpxdata:hr"))==0:
            hr = ''
        else:
            hr = float(trkpt.getElementsByTagName("gpxdata:hr")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("gpxdata:cadence"))==0:
            cad = ''
        else:
            cad = float(trkpt.getElementsByTagName("gpxdata:cadence")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("gpxdata:temp"))==0:
            temp = ''
        else:
            temp = float(trkpt.getElementsByTagName("gpxdata:temp")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("gpxdata:distance"))==0:
            dist = ''
        else:
            dist = float(trkpt.getElementsByTagName("gpxdata:distance")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("gpxdata:power"))==0:
            pwr = ''
        else:
            pwr = float(trkpt.getElementsByTagName("gpxdata:power")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("gpxdata:speed"))==0:
            spd = ''
        else:
            spd = float(trkpt.getElementsByTagName("gpxdata:speed")[0].firstChild.nodeValue)

        timeoffset = to_dt(time)
        
        if (pwr!='') and (alt!=''):
            list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp, fichero])
    
    df_prueba, df_prueba_tramos = enriquecimiento(list_track)

    result_list.append(df_prueba)
    result_list_tramos.append(df_prueba_tramos)
    
    
    print ("Puntos cargados del fichero %s: %d" %(fichero, count))

    
result = pd.concat(result_list)
result_tramos = pd.concat(result_list_tramos)

print ("Puntos cargados: %d" %len(list_track))
result.to_excel('Procesado.xlsx')
result_tramos.to_excel('Procesado_Tramos.xlsx')
print('finish')

Se inicia el procesado de 20180225_LEVS_195_90_R_E.gpx


ipynb.fs.full.Weather_History:150: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Entramos en el While1 5073
Salimos del While
tramos : 138 continuos y 708 de 100m
Puntos cargados del fichero 20180225_LEVS_195_90_R_E.gpx: 5075
Se inicia el procesado de 20180316_LEVS_195_90_R_E.gpx
Entramos en el While1 5279
Salimos del While
tramos : 88 continuos y 711 de 100m
Puntos cargados del fichero 20180316_LEVS_195_90_R_E.gpx: 5279
Se inicia el procesado de 20180318_LEVS_195_90_R_E.gpx


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:202: RuntimeWarning: divide by zero encountered in true_divide


Entramos en el While1 5916
Salimos del While
tramos : 147 continuos y 790 de 100m
Puntos cargados del fichero 20180318_LEVS_195_90_R_E.gpx: 5916
Se inicia el procesado de 20180322_LEVS_195_90_R_E.gpx
Entramos en el While1 5840
Salimos del While
tramos : 117 continuos y 803 de 100m
Puntos cargados del fichero 20180322_LEVS_195_90_R_E.gpx: 5840
Se inicia el procesado de 20180329_LEVD_195_90_R_E.gpx
Entramos en el While1 6516


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars


Salimos del While
tramos : 347 continuos y 878 de 100m
Puntos cargados del fichero 20180329_LEVD_195_90_R_E.gpx: 6516
Se inicia el procesado de 20180406_LEVS_195_90_R_E.gpx
Entramos en el While1 7586
Salimos del While
tramos : 78 continuos y 1057 de 100m
Puntos cargados del fichero 20180406_LEVS_195_90_R_E.gpx: 7586
Se inicia el procesado de 20180415_LEVS_195_90_R_E.gpx
Entramos en el While1 7832
Salimos del While
tramos : 132 continuos y 1077 de 100m
Puntos cargados del fichero 20180415_LEVS_195_90_R_E.gpx: 7832
Se inicia el procesado de 20180422_LEVS_195_90_R_E.gpx
Entramos en el While1 4383
Salimos del While
tramos : 79 continuos y 596 de 100m
Puntos cargados del fichero 20180422_LEVS_195_90_R_E.gpx: 4383
Se inicia el procesado de 20180429_LEMG_195_90_R_C.gpx
Entramos en el While1 6316
Salimos del While
tramos : 123 continuos y 887 de 100m
Puntos cargados del fichero 20180429_LEMG_195_90_R_C.gpx: 6316
Se inicia el procesado de 20180526_LEVS_195_90_C_E.gpx
Entramos en el While1 7185


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: RuntimeWarning: divide by zero encountered in true_divide


Salimos del While
tramos : 161 continuos y 986 de 100m
Puntos cargados del fichero 20180526_LEVS_195_90_C_E.gpx: 7185
Se inicia el procesado de 20180527_LEVS_195_90_C_E.gpx
Entramos en el While1 7206
Salimos del While
tramos : 167 continuos y 986 de 100m
Puntos cargados del fichero 20180527_LEVS_195_90_C_E.gpx: 7206
Puntos cargados: 7206
finish
